<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/ESD_Fire_Alarm_System_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emergency Shutdown (ESD) Fire Alarm Systems - Tutorial

## Learning Objectives

In this tutorial, you will learn:

1. **Theory of ESD Systems**: Understanding fire detection, alarm voting logic, and safety principles
2. **Alarm Voting Patterns**: 1oo1, 2oo2, 2oo3 configurations and their applications
3. **Blowdown Dynamics**: Transient simulation of emergency depressurization
4. **Emissions Calculations**: Heat release and CO₂ emissions from flare systems
5. **NeqSim Implementation**: Practical coding examples using NeqSim process simulation

**Prerequisites**: Basic knowledge of Java, process safety, and thermodynamics

**Author**: NeqSim Development Team  
**Date**: November 2025  
**Version**: 1.0

In [1]:
%%capture
! apt update -q
! apt-get install -q openjdk-25-jdk-headless -q
!pip install neqsim

---
## Part 1: Theory of ESD Fire Alarm Systems

### 1.1 What is an ESD System?

An **Emergency Shutdown (ESD) system** is a Safety Instrumented System (SIS) designed to bring a process to a safe state when hazardous conditions are detected. ESD systems are critical for:

- **Fire Protection**: Detecting and responding to fire events
- **Overpressure Protection**: Preventing vessel rupture
- **Toxic Release Prevention**: Minimizing exposure to hazardous materials
- **Equipment Protection**: Preventing damage to expensive equipment

### 1.2 Fire Detection Methods

Fire detectors use various technologies:

| Type | Detection Method | Application | Response Time |
|------|-----------------|-------------|---------------|
| **Heat Detectors** | Temperature rise | Fixed temperature or rate-of-rise | 30-60 seconds |
| **Smoke Detectors** | Particulate matter | Optical or ionization | 10-30 seconds |
| **Flame Detectors** | IR/UV radiation | Hydrocarbon fires | 3-5 seconds |
| **Gas Detectors** | Combustible gases | LEL (Lower Explosive Limit) | 5-15 seconds |

### 1.3 Why Use Voting Logic?

**Spurious Trip Problem**: A single faulty detector could shut down an entire process, causing:
- Lost production (millions of dollars)
- Equipment thermal/mechanical stress
- Unnecessary flaring and emissions

**Solution**: Use **voting logic** with multiple independent detectors.

### 1.4 Common Voting Configurations

#### **1oo1 (1 out of 1)**
```
Single detector → ESD activates
```
- ✅ Simple, fast response
- ❌ High spurious trip rate
- **Use**: Small, non-critical systems

#### **2oo2 (2 out of 2)**
```
BOTH detectors must activate → ESD
```
- ✅ Eliminates spurious trips
- ❌ System fails if one detector fails (dangerous)
- **Use**: Where spurious trips are very costly

#### **2oo3 (2 out of 3)** ⭐ Industry Standard
```
ANY TWO of three detectors → ESD
```
- ✅ Spurious trip protection
- ✅ Tolerates one detector failure
- ✅ High reliability and availability
- **Use**: Critical safety systems (oil & gas, petrochemical)

### 1.5 Safety Integrity Level (SIL)

Fire and gas systems are typically rated:
- **SIL 2**: 99% - 99.9% probability of functioning on demand
- **SIL 3**: 99.9% - 99.99% probability (critical applications)

Voting logic improves SIL rating by increasing reliability.

---
## Part 2: Blowdown and Flare Systems

### 2.1 What is a Blowdown System?

When ESD activates, the process must be safely depressurized:

1. **Isolation**: Inlet valves close (stop feed)
2. **Depressurization**: Blowdown valve opens
3. **Flaring**: Gas is safely combusted at flare

```
┌──────────────┐
│  Separator   │
│   50 bara    │
└──────┬───────┘
       │
       ├──────────► To Process (normal)
       │
       │  (ESD activates)
       ├──────────► Blowdown Valve
                           │
                    Orifice (flow control)
                           │
                      Flare Stack
                      (1.5 bara)
```

### 2.2 Blowdown Valve Characteristics

- **Normally Closed**: Fail-safe position
- **Opening Time**: 3-10 seconds (prevents water hammer)
- **Cv Rating**: Sized for required depressurization rate
- **Latching**: Stays open until manual reset

### 2.3 Depressurization Rate

API 521 guidelines for depressurization time:

| Scenario | Target Time | Rate |
|----------|-------------|------|
| **Fire case** | 15 minutes | Fast |
| **Overpressure** | As needed | Very fast |
| **Toxic release** | 5-10 minutes | Fast |

Depressurization rate decreases as pressure drops:

$$\dot{m} \propto \sqrt{\Delta P}$$

Where:
- $\dot{m}$ = mass flow rate through orifice
- $\Delta P$ = pressure differential across orifice

### 2.4 Flare Heat Release Calculation

**Heat Release Rate**:
$$Q = \dot{m} \times LCV$$

Where:
- $Q$ = heat release rate (W)
- $\dot{m}$ = mass flow rate (kg/s)
- $LCV$ = Lower Calorific Value (J/kg)

**Cumulative Heat**:
$$Q_{total} = \int_0^t Q(t) \, dt$$

### 2.5 CO₂ Emissions Calculation

For complete combustion of hydrocarbons:

$$C_xH_y + \left(x + \frac{y}{4}\right)O_2 \rightarrow xCO_2 + \frac{y}{2}H_2O$$

**CO₂ Emission Rate**:
$$\dot{m}_{CO_2} = \sum_{i} n_i \times N_{C,i} \times MW_{CO_2}$$

Where:
- $n_i$ = molar flow of component $i$ (mol/s)
- $N_{C,i}$ = number of carbon atoms in component $i$
- $MW_{CO_2}$ = 44.01 g/mol

**Cumulative Emissions**:
$$m_{CO_2,total} = \int_0^t \dot{m}_{CO_2}(t) \, dt$$

In [ ]:
import neqsim.thermo.system.SystemInterface;
import neqsim.thermo.system.SystemSrkEos;
import neqsim.process.equipment.separator.Separator;
import neqsim.process.equipment.stream.Stream;
import neqsim.process.equipment.splitter.Splitter;
import neqsim.process.equipment.valve.BlowdownValve;
import neqsim.process.equipment.diffpressure.Orifice;
import neqsim.process.equipment.flare.Flare;
import neqsim.process.measurementdevice.FireDetector;
import neqsim.process.alarm.AlarmConfig;

---
## Part 4: Example 1 - Basic Fire Detector Setup

### 4.1 Creating Fire Detectors

Let's create two fire detectors for a separator area:

In [ ]:
// Create fire detectors with location information
FireDetector fireDetector1 = new FireDetector("FD-101", "Separator Area - North");
FireDetector fireDetector2 = new FireDetector("FD-102", "Separator Area - South");

// Configure detection parameters
fireDetector1.setDetectionThreshold(0.5);  // 50% signal level
fireDetector1.setDetectionDelay(1.0);      // 1 second confirmation

fireDetector2.setDetectionThreshold(0.5);
fireDetector2.setDetectionDelay(1.0);

System.out.println("Fire Detector 1: " + fireDetector1.getName());
System.out.println("  Location: " + fireDetector1.getLocation());
System.out.println("  Threshold: " + fireDetector1.getDetectionThreshold());
System.out.println("  Status: " + (fireDetector1.isFireDetected() ? "FIRE" : "NORMAL"));
System.out.println();

System.out.println("Fire Detector 2: " + fireDetector2.getName());
System.out.println("  Location: " + fireDetector2.getLocation());
System.out.println("  Status: " + (fireDetector2.isFireDetected() ? "FIRE" : "NORMAL"));

### 4.2 Configuring Alarms

Configure alarm settings for the detectors:

In [ ]:
// Create alarm configuration
AlarmConfig fireAlarmConfig = AlarmConfig.builder()
    .highLimit(0.5)           // Alarm at 50% signal
    .delay(1.0)               // 1 second delay before alarm
    .deadband(0.1)            // 10% deadband for alarm reset
    .unit("binary")
    .build();

// Apply alarm configuration
fireDetector1.setAlarmConfig(fireAlarmConfig);
fireDetector2.setAlarmConfig(fireAlarmConfig);

System.out.println("Alarm Configuration:");
System.out.println("  High Limit: " + fireAlarmConfig.getHighLimit());
System.out.println("  Delay: " + fireAlarmConfig.getDelay() + " seconds");
System.out.println("  Deadband: " + fireAlarmConfig.getDeadband());

### 4.3 Simulating Fire Detection

Simulate a fire event:

In [ ]:
System.out.println("\n=== SIMULATING FIRE EVENT ===");
System.out.println("\nInitial State:");
System.out.println("FD-101: " + (fireDetector1.isFireDetected() ? "FIRE DETECTED" : "NORMAL"));
System.out.println("FD-102: " + (fireDetector2.isFireDetected() ? "FIRE DETECTED" : "NORMAL"));

// First detector activates
System.out.println("\n>>> FIRE DETECTOR FD-101 ACTIVATES <<<");
fireDetector1.detectFire();
System.out.println("FD-101: " + (fireDetector1.isFireDetected() ? "FIRE DETECTED" : "NORMAL"));
System.out.println("FD-102: " + (fireDetector2.isFireDetected() ? "FIRE DETECTED" : "NORMAL"));

// Count active alarms
int activeAlarms = (fireDetector1.isFireDetected() ? 1 : 0) +
                   (fireDetector2.isFireDetected() ? 1 : 0);
System.out.println("Active alarms: " + activeAlarms + " of 2");

// Second detector activates
System.out.println("\n>>> FIRE DETECTOR FD-102 ACTIVATES <<<");
fireDetector2.detectFire();
System.out.println("FD-101: " + (fireDetector1.isFireDetected() ? "FIRE DETECTED" : "NORMAL"));
System.out.println("FD-102: " + (fireDetector2.isFireDetected() ? "FIRE DETECTED" : "NORMAL"));

activeAlarms = (fireDetector1.isFireDetected() ? 1 : 0) +
               (fireDetector2.isFireDetected() ? 1 : 0);
System.out.println("Active alarms: " + activeAlarms + " of 2");
System.out.println("\n>>> ESD SHOULD ACTIVATE (2-out-of-2 voting satisfied) <<<");

---
## Part 5: Example 2 - Complete ESD System with Process Simulation

### 5.1 Process Description

We'll simulate a high-pressure separator with:
- **Operating Pressure**: 50 bara
- **Gas Flow**: 10,000 kg/hr (methane-rich mixture)
- **Fire Detectors**: 2-out-of-2 voting
- **Blowdown Valve**: 5-second opening time
- **Flare Header**: 1.5 bara

### 5.2 Create the Thermodynamic System

In [ ]:
// Create rich gas mixture (typical separator gas)
SystemInterface separatorGas = new SystemSrkEos(298.15, 50.0);
separatorGas.addComponent("nitrogen", 1.0);
separatorGas.addComponent("methane", 85.0);
separatorGas.addComponent("ethane", 10.0);
separatorGas.addComponent("propane", 3.0);
separatorGas.addComponent("n-butane", 1.0);
separatorGas.setMixingRule(2);  // Classic mixing rule

// Create feed stream
Stream feedStream = new Stream("Feed", separatorGas);
feedStream.setFlowRate(10000.0, "kg/hr");
feedStream.setPressure(50.0, "bara");
feedStream.setTemperature(25.0, "C");

System.out.println("Feed Stream Properties:");
System.out.println("  Flow Rate: " + feedStream.getFlowRate("kg/hr") + " kg/hr");
System.out.println("  Pressure: " + feedStream.getPressure("bara") + " bara");
System.out.println("  Temperature: " + feedStream.getTemperature("C") + " °C");

### 5.3 Build the Process Equipment

In [ ]:
// Initialize feed stream
feedStream.run();

// Separator
Separator separator = new Separator("HP Separator", feedStream);
separator.setCalculateSteadyState(true);  // Start in steady-state mode
separator.run();

// Separator gas outlet
Stream separatorGasOut = new Stream("Sep Gas Out", separator.getGasOutStream());
separatorGasOut.run();

// Splitter: routes gas to process OR blowdown
Splitter gasSplitter = new Splitter("Gas Splitter", separatorGasOut, 2);
gasSplitter.setSplitFactors(new double[] {1.0, 0.0});  // Initially: all to process
gasSplitter.run();

Stream processStream = new Stream("To Process", gasSplitter.getSplitStream(0));
Stream blowdownStream = new Stream("To Blowdown", gasSplitter.getSplitStream(1));
processStream.run();
blowdownStream.run();

System.out.println("\nProcess Equipment Configuration:");
System.out.println("  Separator: " + separator.getName());
System.out.println("  Gas Splitter: " + gasSplitter.getName());
System.out.println("  Process flow: " + processStream.getFlowRate("kg/hr") + " kg/hr");
System.out.println("  Blowdown flow: " + blowdownStream.getFlowRate("kg/hr") + " kg/hr");

### 5.4 Configure Blowdown System

In [ ]:
// Blowdown valve (normally closed)
BlowdownValve bdValve = new BlowdownValve("BD-101", blowdownStream);
bdValve.setOpeningTime(5.0);  // 5 seconds to fully open
bdValve.setCv(200.0);         // Flow coefficient
bdValve.run();

Stream bdValveOutlet = new Stream("BD Valve Outlet", bdValve.getOutletStream());
bdValveOutlet.run();

// Blowdown orifice for flow control (ISO 5167)
// Parameters: name, pipe_dia, orifice_dia, P_upstream, P_downstream, Cd
Orifice bdOrifice = new Orifice("BD Orifice", 0.45, 0.18, 50.0, 1.5, 0.61);
bdOrifice.setInletStream(bdValveOutlet);

Stream toFlare = new Stream("To Flare", bdOrifice.getOutletStream());

// Flare
Flare flare = new Flare("Blowdown Flare", toFlare);
flare.setFlameHeight(50.0);        // 50 m flame height
flare.setRadiantFraction(0.20);    // 20% radiant heat
flare.setTipDiameter(0.8);         // 0.8 m tip diameter
flare.resetCumulative();           // Reset counters

System.out.println("\nBlowdown System Configuration:");
System.out.println("  BD Valve: " + bdValve.getName());
System.out.println("    Opening Time: " + bdValve.getOpeningTime() + " seconds");
System.out.println("    Cv: " + bdValve.getCv());
System.out.println("    Status: " + (bdValve.isActivated() ? "ACTIVATED" : "NOT ACTIVATED"));
System.out.println("  BD Orifice: ISO 5167 flow control");
System.out.println("    Pipe diameter: 0.45 m");
System.out.println("    Orifice diameter: 0.18 m");
System.out.println("    Beta ratio: 0.40");
System.out.println("  Flare: " + flare.getName());
System.out.println("    Flame height: 50 m");
System.out.println("    Header pressure: 1.5 bara");

### 5.5 Implement 2-out-of-2 Voting Logic

In [ ]:
// Create fire detectors
FireDetector fd1 = new FireDetector("FD-101", "Separator North");
FireDetector fd2 = new FireDetector("FD-102", "Separator South");

// Configure alarms
AlarmConfig alarmCfg = AlarmConfig.builder()
    .highLimit(0.5)
    .delay(1.0)
    .deadband(0.1)
    .unit("binary")
    .build();

fd1.setAlarmConfig(alarmCfg);
fd2.setAlarmConfig(alarmCfg);

System.out.println("\n=== ESD VOTING LOGIC ===");
System.out.println("Configuration: 2-out-of-2 voting");
System.out.println("  Fire Detector 1: " + fd1.getName());
System.out.println("  Fire Detector 2: " + fd2.getName());
System.out.println("  ESD Trigger: BOTH detectors must activate");

// Helper method for voting logic
public static boolean evaluateESD(FireDetector fd1, FireDetector fd2) {
    int activeAlarms = (fd1.isFireDetected() ? 1 : 0) + (fd2.isFireDetected() ? 1 : 0);
    return activeAlarms >= 2;
}

---
## Part 6: Dynamic Blowdown Simulation

### 6.1 Simulation Phases

Our simulation will have 4 phases:

1. **Normal Operation** (t=0-5s): Both detectors normal
2. **First Alarm** (t=5-10s): FD-101 activates, ESD does NOT trigger
3. **ESD Activation** (t=10s): FD-102 activates, ESD triggers
4. **Blowdown** (t=10-30s): Dynamic depressurization with emissions tracking

### 6.2 Run the Simulation

In [ ]:
import java.util.UUID;

System.out.println("\n╔════════════════════════════════════════════════════════════════╗");
System.out.println("║           DYNAMIC BLOWDOWN SIMULATION                         ║");
System.out.println("╚════════════════════════════════════════════════════════════════╝\n");

// Simulation parameters
double timeStep = 1.0;          // 1 second time steps
double totalTime = 30.0;        // 30 seconds total
double firstAlarmTime = 5.0;    // First alarm at 5s
double esdActivationTime = 10.0; // ESD at 10s

// Results table header
System.out.println("Time | FD-101 | FD-102 | Alarms | ESD    | BD% | Flare MW | CO₂ kg/s | Cumul GJ | Cumul CO₂");
System.out.println("-----|--------|--------|--------|--------|-----|----------|----------|----------|----------");

for (double time = 0.0; time <= totalTime; time += timeStep) {

    // PHASE 2: Activate first fire detector at t=5s
    if (time == firstAlarmTime && !fd1.isFireDetected()) {
        fd1.detectFire();
        System.out.println("\n>>> FIRE DETECTOR FD-101 ACTIVATES <<<\n");
    }

    // PHASE 3: Activate second detector at t=10s
    if (time == esdActivationTime && !fd2.isFireDetected()) {
        fd2.detectFire();
        System.out.println("\n>>> FIRE DETECTOR FD-102 ACTIVATES <<<");
        System.out.println(">>> TWO ALARMS - ESD ACTIVATED <<<\n");

        // Activate ESD
        bdValve.activate();
        gasSplitter.setSplitFactors(new double[] {0.0, 1.0});  // Redirect to blowdown
        separator.setCalculateSteadyState(false);  // Switch to dynamic mode
    }

    // Run process equipment
    if (separator.getCalculateSteadyState()) {
        separator.run();
    } else {
        separator.runTransient(timeStep, UUID.randomUUID());
    }

    separatorGasOut.run();
    gasSplitter.run();
    blowdownStream.run();

    if (time >= esdActivationTime && bdValve.isActivated()) {
        bdValve.runTransient(timeStep, UUID.randomUUID());
    } else {
        bdValve.run();
    }

    bdValveOutlet.run();
    bdOrifice.runTransient(timeStep, UUID.randomUUID());
    toFlare.run();
    flare.run();
    flare.updateCumulative(timeStep);

    // Count active alarms
    int activeAlarms = (fd1.isFireDetected() ? 1 : 0) + (fd2.isFireDetected() ? 1 : 0);
    boolean esdActive = evaluateESD(fd1, fd2);

    // Print results every 2 seconds
    if (time % 2.0 == 0.0 || time == firstAlarmTime || time == esdActivationTime) {
        System.out.printf("%4.0f | %6s | %6s | %6d | %6s | %3.0f | %8.2f | %8.3f | %8.2f | %9.1f%n",
            time,
            fd1.isFireDetected() ? "FIRE" : "OK",
            fd2.isFireDetected() ? "FIRE" : "OK",
            activeAlarms,
            esdActive ? "YES" : "NO",
            bdValve.getPercentValveOpening(),
            flare.getHeatDuty("MW"),
            flare.getCO2Emission("kg/sec"),
            flare.getCumulativeHeatReleased("GJ"),
            flare.getCumulativeCO2Emission("kg")
        );
    }
}

### 6.3 Summary Results

In [ ]:
System.out.println("\n╔════════════════════════════════════════════════════════════════╗");
System.out.println("║                    SIMULATION SUMMARY                          ║");
System.out.println("╚════════════════════════════════════════════════════════════════╝\n");

System.out.println("=== ESD SYSTEM PERFORMANCE ===");
System.out.println("Fire Detector 1: " + fd1.toString());
System.out.println("Fire Detector 2: " + fd2.toString());
System.out.println("\nVoting Logic: 2-out-of-2");
System.out.println("  First alarm (t=5s): ESD did NOT activate ✓");
System.out.println("  Second alarm (t=10s): ESD activated ✓");

System.out.println("\n=== BLOWDOWN VALVE PERFORMANCE ===");
System.out.println("Valve: " + bdValve.getName());
System.out.println("  Final Opening: " + String.format("%.1f%%", bdValve.getPercentValveOpening()));
System.out.println("  Opening Time: 5 seconds");
System.out.println("  Status: " + (bdValve.isActivated() ? "ACTIVATED" : "NOT ACTIVATED"));

System.out.println("\n=== FLARE EMISSIONS SUMMARY ===");
System.out.println("Total gas burned: " + String.format("%.1f kg", flare.getCumulativeGasBurned("kg")));
System.out.println("                  " + String.format("%.2f tonnes", flare.getCumulativeGasBurned("tonnes")));
System.out.println();
System.out.println("Total heat released: " + String.format("%.2f GJ", flare.getCumulativeHeatReleased("GJ")));
System.out.println("                     " + String.format("%.2f MWh", flare.getCumulativeHeatReleased("GJ") * 0.2778));
System.out.println("                     " + String.format("%.2f MMBtu", flare.getCumulativeHeatReleased("MMBtu")));
System.out.println();
System.out.println("Total CO₂ emissions: " + String.format("%.1f kg", flare.getCumulativeCO2Emission("kg")));
System.out.println("                     " + String.format("%.2f tonnes", flare.getCumulativeCO2Emission("kg") / 1000.0));

System.out.println("\n=== ENVIRONMENTAL IMPACT ===");
double co2Tonnes = flare.getCumulativeCO2Emission("kg") / 1000.0;
System.out.println("CO₂ equivalent to:");
System.out.println("  " + String.format("%.1f", co2Tonnes * 0.42) + " passenger cars driven for 1 year");
System.out.println("  " + String.format("%.0f", co2Tonnes * 121.6) + " gallons of gasoline consumed");
System.out.println("  " + String.format("%.1f", co2Tonnes * 1.12) + " metric tons of waste recycled instead of landfilled");

System.out.println("\n╔════════════════════════════════════════════════════════════════╗");
System.out.println("║              SIMULATION COMPLETED SUCCESSFULLY                 ║");
System.out.println("╚════════════════════════════════════════════════════════════════╝");

---
## Part 7: Advanced Topic - 2-out-of-3 Voting

### 7.1 Why 2oo3 Instead of 2oo2?

**Scenario**: What if one detector fails?

| Configuration | Failed Detector | Fire Detected | ESD Activates? | Safe? |
|---------------|----------------|---------------|----------------|-------|
| **2oo2** | FD-1 failed | FD-2 detects | ❌ NO | ❌ Dangerous! |
| **2oo3** | FD-1 failed | FD-2, FD-3 detect | ✅ YES | ✅ Safe |

**Conclusion**: 2oo3 provides redundancy against detector failures.

### 7.2 Implementing 2oo3 Voting

In [ ]:
// Create three fire detectors
FireDetector detector1 = new FireDetector("FD-201", "Zone A");
FireDetector detector2 = new FireDetector("FD-202", "Zone B");
FireDetector detector3 = new FireDetector("FD-203", "Zone C");

// Configure alarms (same config for all)
AlarmConfig cfg = AlarmConfig.builder().highLimit(0.5).delay(0.5).unit("binary").build();
detector1.setAlarmConfig(cfg);
detector2.setAlarmConfig(cfg);
detector3.setAlarmConfig(cfg);

System.out.println("=== 2-OUT-OF-3 VOTING DEMONSTRATION ===");
System.out.println("\nDetector Configuration:");
System.out.println("  " + detector1.getName() + " @ " + detector1.getLocation());
System.out.println("  " + detector2.getName() + " @ " + detector2.getLocation());
System.out.println("  " + detector3.getName() + " @ " + detector3.getLocation());

// Voting logic function
public static int countActiveAlarms(FireDetector... detectors) {
    int count = 0;
    for (FireDetector d : detectors) {
        if (d.isFireDetected()) count++;
    }
    return count;
}

public static boolean evaluate2oo3(FireDetector d1, FireDetector d2, FireDetector d3) {
    return countActiveAlarms(d1, d2, d3) >= 2;
}

// Test scenarios
System.out.println("\n=== TESTING SCENARIOS ===");

// Scenario 1: No alarms
System.out.println("\nScenario 1: No alarms");
int alarms = countActiveAlarms(detector1, detector2, detector3);
System.out.println("  Active: " + alarms + " → ESD: " + (alarms >= 2 ? "YES" : "NO"));

// Scenario 2: One alarm
System.out.println("\nScenario 2: One alarm (FD-201)");
detector1.detectFire();
alarms = countActiveAlarms(detector1, detector2, detector3);
System.out.println("  Active: " + alarms + " → ESD: " + (alarms >= 2 ? "YES" : "NO"));

// Scenario 3: Two alarms (SHOULD TRIGGER)
System.out.println("\nScenario 3: Two alarms (FD-201 + FD-202)");
detector2.detectFire();
alarms = countActiveAlarms(detector1, detector2, detector3);
System.out.println("  Active: " + alarms + " → ESD: " + (alarms >= 2 ? "YES ✓" : "NO"));

// Scenario 4: All three alarms
System.out.println("\nScenario 4: All three alarms");
detector3.detectFire();
alarms = countActiveAlarms(detector1, detector2, detector3);
System.out.println("  Active: " + alarms + " → ESD: " + (alarms >= 2 ? "YES ✓" : "NO"));

// Scenario 5: One detector resets
System.out.println("\nScenario 5: Reset one detector (FD-203)");
detector3.reset();
alarms = countActiveAlarms(detector1, detector2, detector3);
System.out.println("  Active: " + alarms + " → ESD: " + (alarms >= 2 ? "MAINTAINED ✓" : "NO"));

### 7.3 Reliability Comparison

Assuming each detector has:
- **Availability**: 99% (fails 1% of time)
- **False alarm rate**: 0.1% per year

#### Spurious Trip Rate:

**1oo1**: $P_{spurious} = 0.001 = 0.1\%$

**2oo2**: $P_{spurious} = 0.001^2 = 0.0001\% $ ✓ Very low

**2oo3**: $P_{spurious} = 3 \times 0.001^2 \times 0.999 \approx 0.0003\%$ ✓ Very low

#### Failure to Activate (Dangerous):

**1oo1**: $P_{fail} = 0.01 = 1\%$

**2oo2**: $P_{fail} = 1 - (1-0.01)^2 = 1.99\%$ ❌ Higher!

**2oo3**: $P_{fail} = 3 \times 0.01^2 \times 0.99 + 0.01^3 \approx 0.03\%$ ✅ Best!

**Conclusion**: 2oo3 provides both low spurious trips AND high reliability!

---
## Part 8: Exercises and Challenges

### Exercise 1: Modify Operating Conditions

**Task**: Change the separator pressure from 50 bara to 100 bara and observe:
1. How does blowdown time change?
2. How do flare emissions change?
3. What is the pressure decay rate?

**Hint**: Higher pressure → More gas inventory → Longer blowdown

### Exercise 2: Optimize Orifice Size

**Task**: The current orifice has β = 0.40. Try:
1. β = 0.30 (smaller orifice)
2. β = 0.50 (larger orifice)

**Questions**:
- Which configuration depressurizes fastest?
- What are the trade-offs?
- What happens if β is too large?

### Exercise 3: Gas Composition Effects

**Task**: Change the gas composition to:
- Pure methane (C₁ = 100%)
- Rich gas (C₁ = 70%, C₂ = 15%, C₃ = 10%, C₄ = 5%)

**Observe**:
- How does LCV change?
- How do CO₂ emissions change?
- Why does composition matter?

### Exercise 4: Implement 1oo2 Voting

**Task**: Modify the voting logic to 1oo2 (1 out of 2):
```java
boolean esdActivate = (fd1.isFireDetected() || fd2.isFireDetected());
```

**Questions**:
- When does ESD activate now?
- Is this safer or less safe?
- When might 1oo2 be appropriate?

### Challenge: Design Complete ESD System

**Scenario**: You're designing an ESD system for an offshore platform with:
- 3 separators (each at different pressures)
- 2 fire zones requiring 2oo3 voting each
- 1 common flare header
- Target: Depressurize all to 7 bara within 15 minutes

**Requirements**:
1. Design the voting logic
2. Size the orifices
3. Calculate total emissions
4. Verify API 521 compliance

---
## Part 9: Key Takeaways

### Theory
1. ✅ **Voting logic** prevents spurious trips while maintaining safety
2. ✅ **2oo3 configuration** is industry standard for critical systems
3. ✅ **Blowdown rate** decreases with pressure (∝ √ΔP)
4. ✅ **Emissions** depend on gas composition and LCV

### NeqSim Implementation
1. ✅ **FireDetector** class for binary fire detection
2. ✅ **BlowdownValve** with realistic opening dynamics
3. ✅ **Orifice** for ISO 5167 flow control
4. ✅ **Flare** with cumulative heat and CO₂ tracking
5. ✅ **Dynamic simulation** with transient behavior

### Best Practices
1. ✅ Always use **voting logic** for critical safety systems
2. ✅ Configure **alarm delays** to prevent nuisance trips
3. ✅ Implement **safety latching** (manual reset required)
4. ✅ Track **cumulative emissions** for regulatory compliance
5. ✅ Test systems thoroughly with realistic scenarios

---

## References and Further Reading

### Standards
- **API RP 521**: Guide for Pressure-Relieving and Depressuring Systems
- **IEC 61508**: Functional Safety of Electrical/Electronic/Programmable Electronic Safety-related Systems
- **IEC 61511**: Functional Safety - Safety Instrumented Systems for the Process Industry Sector
- **ISO 5167**: Measurement of fluid flow by means of pressure differential devices

### NeqSim Documentation
- **ESD Blowdown System**: `docs/ESD_BLOWDOWN_SYSTEM.md`
- **Fire Alarm System**: `docs/wiki/esd_fire_alarm_system.md`
- **Process Control**: `docs/wiki/process_control.md`
- **Getting Started**: `docs/wiki/getting_started.md`

### Online Resources
- NeqSim Homepage: https://equinor.github.io/neqsim/
- NeqSim GitHub: https://github.com/equinor/neqsim
- Process Safety Articles: https://www.aiche.org/ccps

---

## Appendix: Common Troubleshooting

### Issue 1: Flare shows zero emissions
**Solution**: Ensure blowdown stream has flow:
- Check splitter split factors
- Verify BD valve is activated
- Check orifice sizing

### Issue 2: Valve doesn't open
**Solution**:
- Call `bdValve.activate()` before transient simulation
- Use `runTransient()` not `run()` for dynamic behavior

### Issue 3: Negative flow rates
**Solution**:
- Check pressure gradients (downstream < upstream)
- Verify thermodynamic consistency
- Ensure proper stream initialization

### Issue 4: Simulation crashes
**Solution**:
- Reduce time step size
- Check for zero flow conditions
- Verify equipment sequencing

---

**End of Tutorial**

*For questions or support, please visit the NeqSim community forum or submit an issue on GitHub.*